In [15]:
## arguments
pyspark_bin_path = "/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3"
python_path = "/home/pc/TestJupyter/opt/spark-3.3.0/venv-spark/bin/python39"
app_name = "social media"
data_path = "/home/pc/data/parsed_data/4preview-data-comment_only.csv"
hdfs_working_path = "hdfs://10.123.51.78:8020/user/social_media/"
malay_stopword_path = "/home/pc/Assignment/node_modules/stopwords-ms/stopwords-ms.json"

from connectPyspark import *
%load_ext autoreload
%autoreload 2
import shutil
import os
import json
import findspark
findspark.init(pyspark_bin_path)
import utilities as u
import pyspark
from pyspark.sql import SparkSession
from LangAgg import *
from pyspark.sql.window import Window
from nltk.corpus import stopwords



## initialize spark session
os.environ["HADOOP_USER_NAME"] = "hdfs"
os.environ["PYSPARK_PYTHON"] = python_path
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.3.0  pyspark-shell'


spark = SparkSession.builder \
    .master("local[*]") \
    .appName(app_name) \
    .getOrCreate()


22/10/09 14:37:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
22/10/09 14:37:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [13]:
import pyspark.sql.functions as f

In [42]:
bc = read_avro("output/4_CommentWithLanguage_COST").filter(f.col("lang") == "Language.CHINESE")
bi = read_avro("output/4_CommentWithLanguage_COST").filter(f.col("lang") == "Language.ENGLISH")
bm = read_avro("output/4_CommentWithLanguage_COST").filter(f.col("lang") == "Language.MALAY")

In [43]:
bc = bc.sample(0.002)

In [44]:
bi = bi.sample(0.0001)

In [45]:
bm = bm.sample(0.0001)

In [50]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)


r = unionAll(bi, bm, bc).drop("lang")
r.write.mode("overwrite").csv("output/sample.csv", header=True)